In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Vision_Pro/

/content/drive/MyDrive/Vision_Pro


In [3]:
!ls

blinders_model.h5  dataset3	     model2.h5		 model.h5
b_model.h5	   dataset3_2.ipynb  model3.h5		 model.ipynb
b_model.ipynb	   dataset4	     model4.h5		 predicted_class_test.ipynb
dataset		   keypoints.json    model5.h5		 real_pic
dataset2	   Model_0.59.ipynb  model_camera.ipynb  Untitled


In [100]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical

# 이미지 크기 및 채널 설정
image_height = 64
image_width = 64
channels = 3

# 클래스 정의
class_names = os.listdir('/content/drive/MyDrive/Vision_Pro/dataset4')
num_classes = len(class_names)

def load_and_label_images(folder_path, label):
    image_paths = [os.path.join(folder_path, image_name) for image_name in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, image_name))]
    images = [load_img(image_path, target_size=(image_height, image_width)) for image_path in image_paths]
    image_arrays = [img_to_array(img) for img in images]
    labels = [label] * len(image_paths)
    return image_arrays, labels

# 훈련 데이터 로드 및 라벨링
X_train = []
y_train = []

for label, class_name in enumerate(class_names):
    folder_path = os.path.join('/content/drive/MyDrive/Vision_Pro/dataset4', class_name)
    images, labels = load_and_label_images(folder_path, label)
    X_train.extend(images)
    y_train.extend(labels)

X_train = np.array(X_train)
y_train = to_categorical(y_train, num_classes)

In [101]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/drive/MyDrive/Vision_Pro/dataset4/'
validation_dir = '/content/drive/MyDrive/Vision_Pro/dataset4/'

# 이미지 데이터 생성기 생성
train_datagen = ImageDataGenerator(rescale=1./255,)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 훈련 데이터 로드
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_height, image_width),
    batch_size=64,
    class_mode='categorical'
)

# 검증 데이터 로드
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(image_height, image_width),
    batch_size=64,
    class_mode='categorical'
)

Found 881 images belonging to 3 classes.
Found 881 images belonging to 3 classes.


In [102]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization

# 모델 정의
model3 = Sequential()
model3.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, channels)))
model3.add(MaxPooling2D((2, 2)))
model3.add(Conv2D(64, (3, 3), activation='relu'))
model3.add(MaxPooling2D((2, 2)))
model3.add(Conv2D(128, (3, 3), activation='relu'))  # 추가된 레이어
model3.add(MaxPooling2D((2, 2)))
model3.add(Flatten())
model3.add(Dense(128, activation='relu'))  # 추가된 레이어
model3.add(Dropout(0.5))
model3.add(BatchNormalization())
model3.add(Dense(num_classes, activation='softmax'))


# 모델 정의 시 sample_weight 매개변수 추가
model3.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [103]:
# 모델 학습
history = model3.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=7,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

Epoch 1/7
14/14 [==============================] - 18s 943ms/step - loss: 1.2204 - accuracy: 0.3950 - val_loss: 1.0804 - val_accuracy: 0.3575
Epoch 2/7
14/14 [==============================] - 17s 1s/step - loss: 1.0299 - accuracy: 0.5006 - val_loss: 1.0555 - val_accuracy: 0.6674
Epoch 3/7
14/14 [==============================] - 14s 1s/step - loss: 0.8975 - accuracy: 0.5653 - val_loss: 1.0412 - val_accuracy: 0.7208
Epoch 4/7
14/14 [==============================] - 18s 1s/step - loss: 0.8119 - accuracy: 0.6493 - val_loss: 1.0367 - val_accuracy: 0.6481
Epoch 5/7
14/14 [==============================] - 15s 1s/step - loss: 0.6951 - accuracy: 0.6947 - val_loss: 1.0213 - val_accuracy: 0.5006
Epoch 6/7
14/14 [==============================] - 14s 1s/step - loss: 0.6322 - accuracy: 0.7457 - val_loss: 1.0037 - val_accuracy: 0.6799
Epoch 7/7
14/14 [==============================] - 13s 945ms/step - loss: 0.5791 - accuracy: 0.7809 - val_loss: 0.9827 - val_accuracy: 0.7662


In [104]:
# 테스트 데이터 로드 및 라벨링
X_test = []
y_test = []

for label, class_name in enumerate(class_names):
    folder_path = os.path.join('/content/drive/MyDrive/Vision_Pro/dataset4', class_name)
    images, labels = load_and_label_images(folder_path, label)
    X_test.extend(images)
    y_test.extend(labels)

X_test = np.array(X_test)
y_test = to_categorical(y_test, num_classes)

# 테스트 데이터의 클래스 분포 확인
unique, counts = np.unique(np.argmax(y_test, axis=1), return_counts=True)
class_distribution = dict(zip(unique, counts))
print("Test Data Class Distribution:", class_distribution)

Test Data Class Distribution: {0: 297, 1: 286, 2: 308}


In [105]:
# 테스트 데이터 로드
test_dir = '/content/drive/MyDrive/Vision_Pro/dataset4'
test_generator = validation_datagen.flow_from_directory(
    test_dir,
    target_size=(image_height, image_width),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # 중요: 테스트셋은 섞지 않도록 설정
)

# 모델 평가
test_loss, test_accuracy = model3.evaluate(test_generator, steps=len(test_generator))
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Found 881 images belonging to 3 classes.
28/28 [==============================] - 5s 164ms/step - loss: 0.9827 - accuracy: 0.7662
Test Loss: 0.9826955795288086
Test Accuracy: 76.62%


In [106]:
from sklearn.metrics import confusion_matrix, classification_report

# 테스트 데이터 로드
X_test = []
y_test = []

for label, class_name in enumerate(class_names):
    folder_path = os.path.join('/content/drive/MyDrive/Vision_Pro/dataset4', class_name)
    images, labels = load_and_label_images(folder_path, label)
    X_test.extend(images)
    y_test.extend(labels)

X_test = np.array(X_test)
y_test = np.array(y_test)  # one-hot 인코딩을 다시 정수로 변환

# 모델 예측
y_pred = model3.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)  # one-hot 인코딩을 다시 정수로 변환

# Confusion Matrix 계산
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Classification Report 출력
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=class_names))


28/28 [==============================] - 3s 84ms/step
Confusion Matrix:
[[  0  42 255]
 [220  21  45]
 [  5 299   4]]

Classification Report:
              precision    recall  f1-score   support

    step_3_3       0.00      0.00      0.00       297
    step_1_3       0.06      0.07      0.06       286
    step_2_3       0.01      0.01      0.01       308

    accuracy                           0.03       891
   macro avg       0.02      0.03      0.03       891
weighted avg       0.02      0.03      0.03       891



In [107]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image

def extract_class_from_filename(filename):
    # 파일 이름에서 클래스를 추출합니다.
    class_name = filename.split('_')[0]  # 파일 이름에서 '_'로 구분된 첫 부분을 클래스로 사용합니다.
    return class_name

def load_and_preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(image_height, image_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # 이미지를 0과 1 사이의 값으로 정규화

    return img_array

# 이미지 폴더 경로
image_folder_path = "/content/drive/MyDrive/Vision_Pro/dataset4/"

# 전체 이미지에 대한 클래스 확인
for class_name in os.listdir(image_folder_path):
    class_path = os.path.join(image_folder_path, class_name)
    if os.path.isdir(class_path):
        print(f"Class: {class_name}")
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            if os.path.isfile(image_path) and image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                # 이미지 로드 및 전처리
                input_image = load_and_preprocess_image(image_path)

                # 모델로 예측
                predictions = model3.predict(input_image)
                predicted_class = np.argmax(predictions)

                # 예측 결과 출력
                print(f"    Image: {image_name}, Predicted Class: class_{predicted_class}")

Class: step_3_3
1/1 [==============================] - 0s 30ms/step
    Image: 20230312_105802.jpg, Predicted Class: class_1
1/1 [==============================] - 0s 27ms/step
    Image: 20230312_105953.jpg, Predicted Class: class_1
1/1 [==============================] - 0s 28ms/step
    Image: depositphotos_84371762-stock-photo-woman-in-yoga-tadasana-pose.jpg, Predicted Class: class_2
1/1 [==============================] - 0s 26ms/step
    Image: iStock-664505904_60d37695c52a6.jpeg, Predicted Class: class_2
1/1 [==============================] - 0s 26ms/step
    Image: woman-yoga-exercices-tadasana-mountain-pose-one-caucasian-woman-exercising-yoga-exercices-tadasana-mountain-pose-silhouette-174251571.jpg, Predicted Class: class_2
1/1 [==============================] - 0s 27ms/step
    Image: istockphoto-543189166-612x612.jpg, Predicted Class: class_2
1/1 [==============================] - 0s 28ms/step
    Image: tadasana.jpg, Predicted Class: class_2
1/1 [============================

In [119]:
# 예측할 이미지의 경로
image_path_to_predict = "/content/drive/MyDrive/Vision_Pro/real_pic/3step.jpg"

# 이미지 로드 및 전처리
input_image = load_and_preprocess_image(image_path_to_predict)

# 모델로 예측
predictions = model3.predict(input_image)
predicted_class = np.argmax(predictions)

# 예측 결과 출력
print(f"Predicted Class: class_{predicted_class}")
print(f"Predictions: {predictions[0]}")
second_highest_prob = np.sort(predictions[0])[-2]
print(f"Second Highest Probability : {second_highest_prob}")

1/1 [==============================] - 0s 94ms/step
Predicted Class: class_1
Predictions: [0.296292   0.38670206 0.31700587]
Second Highest Probability : 0.31700587272644043


In [123]:
class_0_prob = predictions[0][0]
class_1_prob = predictions[0][1]
class_2_prob = predictions[0][2]

if 0.31 <= class_0_prob < 0.35 and class_1_prob < 0.385 and class_2_prob < 0.316:
    predicted_class = "class_0"
elif 0.3 <= class_0_prob < 0.355 and 0.35 <= class_1_prob and 0.26 <= class_2_prob < 0.325:
    predicted_class = "class_1"
elif 0.29 <= class_0_prob < 0.355 and 0.36 <= class_1_prob < 0.39 and 0.28 <= class_2_prob < 0.32:
    predicted_class = "class_2"
else:
    predicted_class = "Unclassified"

print(f"Predicted Class: {predicted_class}")
print(f"Predictions: {predictions[0]}")

Predicted Class: class_2
Predictions: [0.296292   0.38670206 0.31700587]


In [124]:
model3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 31, 31, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_13 (Conv2D)          (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 14, 14, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_14 (Conv2D)          (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 6, 6, 128)        

In [125]:
model_save_path = '/content/drive/MyDrive/Vision_Pro/i_model.h5'
model3.save(model_save_path)
print(f"모델이 {model_save_path}에 저장되었습니다.")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


모델이 /content/drive/MyDrive/Vision_Pro/i_model.h5에 저장되었습니다.
